import the necessary packages]

In [56]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [57]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


In [58]:
!python3 --version

Python 3.8.10


In [59]:
dataset = np.array([15838159496439655, 15838159696040989, 15838159895702853, 15838160095368803, 15838160295031301, 15838160494695483, 15838160694375873, 15838160894041003, 15838161093704287, 15838161293376177, 15838161493039407, 15838161692702571, 15838161892384809, 15838162092047203, 15838162291709467, 15838162491381829, 15838162691046735, 15838162890709833, 15838163090389517, 15838163290052993, 15838163489717045, 15838163689396893, 15838163889060949, 15838164088724445, 15838164288404735, 15838164488068967, 15838164687732665, 15838164887412259, 15838165087077493, 15838165286740897, 15838165486421673, 15838165686087013, 15838165885748919, 15838166085412599, 15838166285073097, 15838166484736693, 15838166684397967, 15838166884062801, 15838167083724955, 15838167283387685, 15838167483048819, 15838167682712365, 15838167882374629, 15838168082036839, 15838168281701869, 15838168481363701, 15838168681026459, 15838168880693583, 15838169080357279, 15838169280019899, 15838169479684875, 15838169679346983, 15838169879010793, 15838170078675035, 15838170278339047, 15838170478004013, 15838170677667013, 15838170877330241, 15838171076995685, 15838171276660203, 15838171476323865, 15838171675987107, 15838171875669953, 15838172075334291, 15838172274998613, 15838172474678861, 15838172674344117, 15838172874005445, 15838173073679307, 15838173273341567, 15838173473006953, 15838173672670431, 15838173872344773, 15838174072009705, 15838174271677595, 15838174471360317, 15838174671023223, 15838174870688177, 15838175070359341, 15838175270023179, 15838175469687233, 15838175669359851, 15838175869023981, 15838176068686109, 15838176268366057, 15838176468029077, 15838176667693831, 15838176867368073, 15838177067031807, 15838177266694967, 15838177466357339, 15838177666018787, 15838177865682663, 15838178065344867, 15838178265006687, 15838178464666861, 15838178664331651, 15838178863997385, 15838179063660633, 15838179263322537])
dataset = dataset * 0.00000001

In [60]:
dataset[0:2]

array([1.58381595e+08, 1.58381597e+08])

In [61]:
start = dataset[0]
end = dataset[-1]
interval = end - start
start, end, interval


(158381594.96439657, 158381792.63322535, 197.66882878541946)

In [62]:
dataset = dataset - start
dataset[0], dataset[-1]

(0.0, 197.66882878541946)

In [63]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = dataset.reshape(-1,1)
dataset = scaler.fit_transform(dataset)
dataset

array([[0.        ],
       [0.01009776],
       [0.02019859],
       [0.03029963],
       [0.04040048],
       [0.05050143],
       [0.06060319],
       [0.07070419],
       [0.08080508],
       [0.09090642],
       [0.10100731],
       [0.11110821],
       [0.12121006],
       [0.13131092],
       [0.14141177],
       [0.15151312],
       [0.1616141 ],
       [0.17171499],
       [0.18181672],
       [0.19191763],
       [0.20201857],
       [0.21212031],
       [0.22222124],
       [0.23232215],
       [0.24242391],
       [0.25252486],
       [0.26262578],
       [0.2727275 ],
       [0.2828285 ],
       [0.2929294 ],
       [0.30303119],
       [0.31313219],
       [0.32323302],
       [0.33333394],
       [0.3434347 ],
       [0.35353561],
       [0.36363641],
       [0.37373739],
       [0.38383823],
       [0.3939391 ],
       [0.40403989],
       [0.4141408 ],
       [0.42424165],
       [0.43434249],
       [0.44444348],
       [0.45454431],
       [0.46464518],
       [0.474

In [64]:
look_back = 50
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [65]:
trainX, trainY = create_dataset(dataset=dataset, look_back=look_back)

In [66]:
trainX[:2],trainY[:2]

(array([[0.        , 0.01009776, 0.02019859, 0.03029963, 0.04040048,
         0.05050143, 0.06060319, 0.07070419, 0.08080508, 0.09090642,
         0.10100731, 0.11110821, 0.12121006, 0.13131092, 0.14141177,
         0.15151312, 0.1616141 , 0.17171499, 0.18181672, 0.19191763,
         0.20201857, 0.21212031, 0.22222124, 0.23232215, 0.24242391,
         0.25252486, 0.26262578, 0.2727275 , 0.2828285 , 0.2929294 ,
         0.30303119, 0.31313219, 0.32323302, 0.33333394, 0.3434347 ,
         0.35353561, 0.36363641, 0.37373739, 0.38383823, 0.3939391 ,
         0.40403989, 0.4141408 , 0.42424165, 0.43434249, 0.44444348,
         0.45454431, 0.46464518, 0.47474627, 0.48484719, 0.49494806],
        [0.01009776, 0.02019859, 0.03029963, 0.04040048, 0.05050143,
         0.06060319, 0.07070419, 0.08080508, 0.09090642, 0.10100731,
         0.11110821, 0.12121006, 0.13131092, 0.14141177, 0.15151312,
         0.1616141 , 0.17171499, 0.18181672, 0.19191763, 0.20201857,
         0.21212031, 0.22222124, 

In [67]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

In [68]:
trainX

array([[[0.        , 0.01009776, 0.02019859, ..., 0.47474627,
         0.48484719, 0.49494806]],

       [[0.01009776, 0.02019859, 0.03029963, ..., 0.48484719,
         0.49494806, 0.50504904]],

       [[0.02019859, 0.03029963, 0.04040048, ..., 0.49494806,
         0.50504904, 0.51514988]],

       ...,

       [[0.46464518, 0.47474627, 0.48484719, ..., 0.93939471,
         0.94949553, 0.95959628]],

       [[0.47474627, 0.48484719, 0.49494806, ..., 0.94949553,
         0.95959628, 0.96969725]],

       [[0.48484719, 0.49494806, 0.50504904, ..., 0.95959628,
         0.96969725, 0.97979827]]])

In [69]:
trainY

array([0.50504904, 0.51514988, 0.5252508 , 0.53535175, 0.54545269,
       0.55555367, 0.56565456, 0.57575545, 0.58585646, 0.59595742,
       0.60605834, 0.61615924, 0.62626112, 0.63636208, 0.64646303,
       0.65656478, 0.66666578, 0.67676658, 0.68686802, 0.69696886,
       0.70706987, 0.71717078, 0.72727224, 0.73737322, 0.74747435,
       0.75757623, 0.76767711, 0.7777781 , 0.78787939, 0.79798032,
       0.80808126, 0.81818263, 0.82828357, 0.83838441, 0.84848615,
       0.85858704, 0.86868801, 0.87878946, 0.88889039, 0.89899128,
       0.90909213, 0.91919294, 0.92929387, 0.93939471, 0.94949553,
       0.95959628, 0.96969725, 0.97979827, 0.98989917])

In [70]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
49/49 - 1s - loss: 0.1017 - accuracy: 0.0000e+00 - 1s/epoch - 26ms/step
Epoch 2/10
49/49 - 0s - loss: 4.7610e-04 - accuracy: 0.0000e+00 - 73ms/epoch - 1ms/step
Epoch 3/10
49/49 - 0s - loss: 2.9190e-04 - accuracy: 0.0000e+00 - 66ms/epoch - 1ms/step
Epoch 4/10
49/49 - 0s - loss: 2.7220e-04 - accuracy: 0.0000e+00 - 68ms/epoch - 1ms/step
Epoch 5/10
49/49 - 0s - loss: 2.4419e-04 - accuracy: 0.0000e+00 - 66ms/epoch - 1ms/step
Epoch 6/10
49/49 - 0s - loss: 2.2972e-04 - accuracy: 0.0000e+00 - 68ms/epoch - 1ms/step
Epoch 7/10
49/49 - 0s - loss: 2.3325e-04 - accuracy: 0.0000e+00 - 65ms/epoch - 1ms/step
Epoch 8/10
49/49 - 0s - loss: 2.1694e-04 - accuracy: 0.0000e+00 - 64ms/epoch - 1ms/step
Epoch 9/10
49/49 - 0s - loss: 1.8739e-04 - accuracy: 0.0000e+00 - 67ms/epoch - 1ms/step
Epoch 10/10
49/49 - 0s - loss: 2.0217e-04 - accuracy: 0.0000e+00 - 64ms/epoch - 1ms/step


In [71]:
inference_model = tf.keras.models.clone_model(model)
inference_model

In [72]:
trainPredict = model.predict(trainX)
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

2/2 [==============================] - 0s 3ms/step
Train Score: 2.61 RMSE
